In [88]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [89]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')



You can see in below a wavelet transformed image that gives clues on facial features such as eyes, nose, lips etc. This along with raw pixel image can be used as an input for our classifier(Proses Ekstraksi Fiture)


In [90]:
import numpy as np
import pywt
import cv2    

def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [91]:
def get_cropped_image_if_2_eyes(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            return roi_color

In [92]:
import glob
path_to_data = glob.glob("./dataset/Koleksi/*.jpg")
path_to_cr_data = "./dataset/Koleksi/cropped/"

In [93]:
path_to_data = "./dataset/Koleksi/"
path_to_cr_data = "./dataset/Koleksi/cropped/"

Mengambul seluruh Folder/GAmbar untuk Digunakan pada proses Cropping

In [94]:
import os
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [95]:
img_dirs

['./dataset/Koleksi/cristiano_ronaldo', './dataset/Koleksi/leo_messi']

Membuat Folder Baru Tempat Penyimpanan Hasil Cropping Wajah

In [96]:
import shutil
if os.path.exists(path_to_cr_data):
     shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

Melakukan Croping Pada Objek

In [97]:
cropped_image_dirs = []
celebrity_file_names_dict = {}
for img_dirs in img_dirs:
    count = 1
    celebrity_name = img_dirs.split('/')[-1]
    celebrity_file_names_dict[celebrity_name] = []
    for entry in os.scandir(img_dirs):
        roi_color = get_cropped_image_if_2_eyes(entry.path)
        if roi_color is not None:
            cropped_folder = path_to_cr_data + celebrity_name
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_image_dirs.append(cropped_folder)
                print("Generating cropped images in folder: ",cropped_folder)
            cropped_file_name = celebrity_name + str(count) + ".png"
            cropped_file_path = cropped_folder + "/" + cropped_file_name
            cv2.imwrite(cropped_file_path, roi_color)
            celebrity_file_names_dict[celebrity_name].append(cropped_file_path)
            count += 1

Generating cropped images in folder:  ./dataset/Koleksi/cropped/cristiano_ronaldo
Generating cropped images in folder:  ./dataset/Koleksi/cropped/leo_messi


In [98]:
class_dict = {}
count = 0
for celebrity_name in celebrity_file_names_dict.keys():
    class_dict[celebrity_name] = count
    count = count + 1
class_dict

{'cristiano_ronaldo': 0, 'leo_messi': 1}

In [99]:
X, y = [], []
for celebrity_name, training_files in celebrity_file_names_dict.items():
    for training_image in training_files:
        img = cv2.imread(training_image)
        scalled_raw_img = cv2.resize(img, (32, 32))
        img_har = w2d(img,'db1',5)
        scalled_img_har = cv2.resize(img_har, (32, 32))
        combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
        X.append(combined_img)
        y.append(class_dict[celebrity_name])
        #final = combined_img.reshape(1,len_image_array).astype(float)  

In [100]:
len(X[0])

4096

In [101]:
combined_img

array([[ 13],
       [ 17],
       [ 22],
       ...,
       [240],
       [254],
       [243]], dtype=uint8)

In [102]:
X = np.array(X).reshape(len(X),4096).astype(float)
X.shape

(56, 4096)

Score Klasifikasi Menggunakan SVM

In [103]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel = 'rbf', C = 10))])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)


0.6428571428571429

In [105]:
len(X_test)

14

from pathlib import Path
feature_path = Path("./static/feature") / (celebrity_name + ".npy")  # e.g., ./static/feature/xxx.npy
np.save(feature_path, X)

Score Klasifikasi Menggunakan DecisionTree


In [106]:


from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree = decision_tree.fit(X_train, y_train)
decision_tree.score(X_test, y_test)

0.6428571428571429

In [107]:
decision_tree.predict(X_test)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [108]:
class_dict1 = {}
count = 0
for img_path1 in sorted(Path("./static/feature").glob("*.jpg")):
    class_dict1[celebrity_name1] = count
    count = count + 1
class_dict

{'cristiano_ronaldo': 0, 'leo_messi': 1}

In [109]:
X, y = [], []
for celebrity_name, training_files in celebrity_file_names_dict.items():
    for training_image in training_files:
        img = cv2.imread(training_image)
        scalled_raw_img = cv2.resize(img, (32, 32))
        img_har = w2d(img,'db1',5)
        scalled_img_har = cv2.resize(img_har, (32, 32))
        combined_img = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
        X.append(combined_img)
        y.append(class_dict[celebrity_name])

Test Menggunakan Gambar Lain

In [110]:
from PIL import Image
from feature_extractor import FeatureExtractor
from pathlib import Path
import numpy as np
for img_path1 in sorted(Path("./static/img").glob("*.jpg")):
    img1= np.array(Image.open(img_path1))
    scalled_raw_img1 = cv2.resize(img1, (32, 32))
    img_har1 = w2d(img1,'db1',5)
    scalled_img_har1 = cv2.resize(img_har1, (32, 32))
    combined_img1 = np.vstack((scalled_raw_img1.reshape(32*32*3,1),scalled_img_har1.reshape(32*32,1)))


    len_image_array = 32*32*3 + 32*32

    final1 = combined_img1.reshape(1,len_image_array).astype(float)
    
    feature_path = Path("./static/feature") / (img_path1.stem + ".npy")  # e.g., ./static/feature/xxx.npy
    np.save(feature_path, final1)

========Testing======

In [111]:
X = np.array(X).reshape(len(X),4096).astype(float)
X.shape

(56, 4096)

In [112]:
len_image_array = 32*32*3 + 32*32

        final = combined_img1.reshape(1,len_image_array).astype(float)
        final1 =decision_tree.predict(final)
        feature_path = Path("./static/feature") / (img_path1.stem + ".npy")  # e.g., ./static/feature/xxx.npy
        np.save(feature_path, final1)

IndentationError: unexpected indent (1830001628.py, line 3)

In [ ]:
from pathlib import Path


In [ ]:
from PIL import Image
from feature_extractor import FeatureExtractor
from pathlib import Path
import numpy as np
if __name__ == '__main__':
    fe = w2d()

    for img_path in sorted(Path("./static/img").glob("*.jpg")):
        print(img_path)  # e.g., ./static/img/xxx.jpg
        feature = fe.extract(img=Image.open(img_path))
        feature_path = Path("./static/feature") / (img_path.stem + ".npy")  # e.g., ./static/feature/xxx.npy
        np.save(feature_path, feature)

In [ ]:
img3 = cv2.imread("./static/img/Kuda.jpg")
scalled_raw_img3 = cv2.resize(img3, (32, 32))
img_har3 = w2d(img3,'db1',5)
scalled_img_har3 = cv2.resize(img_har3, (32, 32))
combined_img3 = np.vstack((scalled_raw_img3.reshape(32*32*3,1),scalled_img_har3.reshape(32*32,1)))


len_image_array = 32*32*3 + 32*32

final = combined_img3.reshape(1,len_image_array).astype(float)

In [ ]:

v = decision_tree.predict(final)


if v == 1 :
    print('Messi')
if v == 0 :
    print('Ronaldo')
else:
    print("Not Match")

Ronaldo


In [ ]:
decision_tree.predict_proba(final)

array([[1., 0.]])

array([[ 39.,  79.,  94., ..., 240., 254., 243.]])

========= HANYA CONTOH TIDAK MASUK PROGRAM ============

In [ ]:
path_to_data1 = "./static/img/"
path_to_cr_data1 = "./static/img/crop/"

import os
img_dirs = []
for entry in os.scandir(path_to_data1):
    if entry.is_dir():
        img_dirs.append(entry.path)


import shutil
if os.path.exists(path_to_cr_data1):
     shutil.rmtree(path_to_cr_data1)
os.mkdir(path_to_cr_data1)




In [ ]:
cropped_image_dirs = []
celebrity_file_names_dict = {}
for img_dirs in img_dirs:
    count = 1
    celebrity_name = img_dirs.split('/')[-1]
    celebrity_file_names_dict[celebrity_name] = []
    for entry in os.scandir(img_dirs):
        roi_color = get_cropped_image_if_2_eyes(entry.path)
        if roi_color is not None:
            cropped_folder = path_to_cr_data + celebrity_name
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_image_dirs.append(cropped_folder)
                print("Generating cropped images in folder: ",cropped_folder)
            cropped_file_name = celebrity_name + str(count) + ".png"
            cropped_file_path = cropped_folder + "/" + cropped_file_name
            cv2.imwrite(cropped_file_path, roi_color)
            celebrity_file_names_dict[celebrity_name].append(cropped_file_path)
            count += 1

Generating cropped images in folder:  ./static/img/crop/test


In [ ]:
import numpy as np
from PIL import Image
from feature_extractor import FeatureExtractor
from datetime import datetime

from pathlib import Path

In [ ]:
features = []
img_paths = []
for feature_path in Path("./static/feature").glob("*.npy"):
    features.append(np.load(feature_path))
    img_paths.append(Path("./static/img") / (feature_path.stem + ".jpg"))
features = np.array(features)

In [ ]:
img_paths

[WindowsPath('static/img/Kuda.jpg'),
 WindowsPath('static/img/mess1.jpg'),
 WindowsPath('static/img/mess2.jpg'),
 WindowsPath('static/img/mess3.jpg'),
 WindowsPath('static/img/mess4.jpg'),
 WindowsPath('static/img/mess5.jpg'),
 WindowsPath('static/img/mess6.jpg'),
 WindowsPath('static/img/mess7.jpg'),
 WindowsPath('static/img/mess8.jpg'),
 WindowsPath('static/img/mess9.jpg'),
 WindowsPath('static/img/roanl3.jpg'),
 WindowsPath('static/img/ronal1.jpg'),
 WindowsPath('static/img/ronal3.jpg'),
 WindowsPath('static/img/ronal4.jpg'),
 WindowsPath('static/img/ronal5.jpg'),
 WindowsPath('static/img/ronal6.jpg'),
 WindowsPath('static/img/ronal7.jpg'),
 WindowsPath('static/img/ronald3.jpg')]

In [ ]:
img = Image.open(file.stream)  # PIL image
uploaded_img_path = "static/uploaded/" + datetime.now().isoformat().replace(":", ".") + "_" + file.filename
img.save(uploaded_img_path)


ValueError: unknown file extension: .926851_

=====TESTING====

In [ ]:
import numpy as np
from PIL import Image
from feature_extractor import FeatureExtractor
from datetime import datetime

from pathlib import Path
import cv2

features = []
img_paths = []
for feature_path in Path("./static/feature").glob("*.npy"):
    features.append(np.load(feature_path))
    img_paths.append(Path("./static/img") / (feature_path.stem + ".jpg"))
features = np.array(features)

In [ ]:
file = cv2.imread("./static/img/Kuda.jpg")

        # Save query image
 # PIL image
scalled_raw_img1 = cv2.resize(file, (32, 32))
uploaded_img_path = "C:/Users/roni/Desktop/TA_project/static/upload" 
file.save(uploaded_img_path)
        
        # Run search
query = fe.w2d(file)
scalled_img_har1 = cv2.resize(query, (32, 32))
        
combined_img1 = np.vstack((scalled_raw_img1.reshape(32*32*3,1),scalled_img_har1.reshape(32*32,1)))
len_image_array = 32*32*3 + 32*32
final1 = combined_img1.reshape(1,len_image_array).astype(float)
dists = np.linalg.norm(features-final1, axis=1)  # L2 distances to features
ids = np.argsort(dists)[:10]  # Top 30 results
scores = [(dists[id], img_paths[id]) for id in ids]


AttributeError: 'numpy.ndarray' object has no attribute 'save'

In [116]:
import numpy as np
import pywt
import cv2    





def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

def extract( img):
    scalled_raw_img3 = cv2.resize(img, (32, 32))
    img_har3 = w2d(img,'db1',5)
    scalled_img_har3 = cv2.resize(img_har3, (32, 32))
    combined_img3 = np.vstack((scalled_raw_img3.reshape(32*32*3,1),scalled_img_har3.reshape(32*32,1)))


    len_image_array = 32*32*3 + 32*32

    final = combined_img3.reshape(1,len_image_array).astype(float)
    feature = decision_tree.predict(final)
    return feature / np.linalg.norm(feature)


In [117]:
from PIL import Image
from feature_extractor import FeatureExtractor
from pathlib import Path
import numpy as np
for img_path1 in sorted(Path("./static/img").glob("*.jpg")):
    feature = extract(np.array(Image.open(img_path1)))
    feature_path = Path("./static/feature") / (img_path1.stem + ".npy")  # e.g., ./static/feature/xxx.npy
    np.save(feature_path, feature)

c:\Users\roni\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
c:\Users\roni\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
c:\Users\roni\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
c:\Users\roni\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
c:\Users\roni\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
c:\Users\roni\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
c:\Users\roni\AppData\Local\Programs\Python\Python37\lib\site-packages\ipyke

In [120]:
# Import the libraries
import matplotlib.pyplot as plt
import numpy as np
# Insert the image query
img = Image.open("C:/Users/roni/Desktop/TA_project/dataset/Koleksi/cristiano_ronaldo/cristiano_ronaldo4.png")
# Extract its features
query = extract(np.array(img))
# Calculate the similarity (distance) between images
dists = np.linalg.norm(features - query, axis=1)
# Extract 30 images that have lowest distance
ids = np.argsort(dists)[:30]
scores = [(dists[id], img_paths[id]) for id in ids]
# Visualize the result
axes=[]
fig=plt.figure(figsize=(8,8))
for a in range(5*6):
    score = scores[a]
    axes.append(fig.add_subplot(5, 6, a+1))
    subplot_title=str(score[0])
    axes[-1].set_title(subplot_title)  
    plt.axis('off')
    plt.imshow(Image.open(score[1]))
fig.tight_layout()
plt.show()

c:\Users\roni\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide


IndexError: index 2722 is out of bounds for axis 0 with size 18